In [ ]:
import numpy as np
import pandas as pd 
import altair as alt
import copy
import re 
import ntpath 
from commons import data_processing
from commons.DataProcessors import msfragger
from commons.APICallers import uniprot

alt.data_transformers.disable_max_rows()

In [ ]:
column_colors = alt.Color('column:N',
    scale=alt.Scale(
        domain=["PGC", "RPLC", "both"],
        range=["#D56E3B", "#58728C", "#AAAAAA"]
    )
)

axis_params = alt.Axis(
    labelFontSize=14,
    labelFontWeight=600,
    labelAngle=0,
    labelFlush=False
)

In [ ]:
# read in data from pax db
# https://pax-db.org/dataset/9606/1502934799/
pax = pd.read_csv('./data/paxdb/9606-WHOLE_ORGANISM-integrated.txt', delimiter='\t', skiprows=range(10))
pax.columns = [c.replace('#', '') for c in pax.columns]

# read in uniprot mappings from paxdb
# https://pax-db.org/download
pax_map = pd.read_csv('./data/paxdb/paxdb-uniprot-links-v4.2.tsv', delimiter='\t', header=None)
pax_map.columns = ['internal_id', 'uniprot_id']

# filter mappings to only human proteins
pax_map_human = pax_map[pax_map.uniprot_id.str.contains('HUMAN')]
pax_human_lookup = dict(zip(pax_map_human.internal_id.tolist(), pax_map_human.uniprot_id.tolist()))


In [ ]:
# map uniprot entries to pax db
pax.loc[:, "uniprot_id"] = pax.string_external_id.map(pax_human_lookup)
pax.loc[:, "rank"] = [i for i in range(1, len(pax)+1)]
pax

In [ ]:
# make a test chart to see if the data is how we expect
alt.Chart(pax).mark_circle().encode(
    x=alt.X('rank:Q', title='paxdb rank',
        axis=axis_params),
    y=alt.Y('log_abundance:Q', title='paxdf abundance',
        axis=axis_params),
    # size='abundance:Q'
).transform_calculate(
    log_abundance='log(datum.abundance)'
)

In [ ]:
# now we read in experimental data
files = data_processing.get_files(r'./data/fractions/', exts=['psm.tsv'])

# use msfragger module to process data
msf = None
for file in files:
    m = msfragger.msf_processor([file])

    # extract fraction and column info
    dirname = ntpath.dirname(file)
    spl = re.split(r'[\\\/]', dirname)
    column, fraction = spl[-2:]
    m.add_special_column('fraction', fraction)
    m.add_special_column('column', column)

    if msf is None:
        msf = m
    else:
        msf.join_processors(m)

# grab data
df = msf.data
msf

In [ ]:
# combine based on which column identified the proteins
exp_proteins = df.groupby(['column', 'entry_name', 'protein_id', 'gene'], as_index=False).count()

# find all proteins identified in both
counts = exp_proteins.protein_id.value_counts()
pairs = counts[counts.values==2].keys()
print(f'{len(counts)} total proteins, {len(pairs)} identified in both columns')

# change column name to 'both'
exp_proteins.loc[(exp_proteins.protein_id.isin(pairs)), 'column'] = 'both'

# remove those duplicates
exp_proteins = exp_proteins.drop_duplicates(['column', 'entry_name'])

# keep only needed rows
exp_proteins = exp_proteins.iloc[:, :4]

In [ ]:
# merge to pax db based on entry name
pax_join = pax.merge(exp_proteins, left_on='uniprot_id', right_on='entry_name')

# calculate log10 abundance
# since it includes values << 10, we will multiply by 1000 for normalization
pax_join.loc[:, 'log10_abundance'] = np.log10(pax_join.abundance*1000)

# normalize
pax_join.loc[:, 'norm10_abundance'] = pax_join.log10_abundance /  np.max(pax_join.log10_abundance)

abund_dots = alt.Chart(pax_join).mark_circle(
    size=10
).encode(
    x=alt.X('rank:Q', title='paxdb rank',
        axis=axis_params),
    y=alt.Y('norm10_abundance:Q', title='paxdf abundance, normalized',
        axis=axis_params),
    # size='abundance:Q',
    color=column_colors,
    column='column:N'
).properties(
    width=100,
    height=300
)

In [ ]:
abund_box = alt.Chart(pax_join, width=53).mark_boxplot(
    size=40
).encode(
    y=alt.Y('norm10_abundance:Q', title='paxdb abundance, normalized',
        axis=axis_params),
    column='column:N',
    color=column_colors
).transform_calculate(
    log_abundance='log(datum.abundance)'
).configure_facet(
    spacing=0
).configure_view(
    stroke=None
)

In [ ]:
abund_dots

In [ ]:
abund_box

In [ ]:
pax_join[pax_join.column.isin(['both', 'RPLC'])].protein_id.to_clipboard(header=False, index=False)

In [ ]:
import seaborn as sns

sns.violinplot(x='column', y='norm10_abundance', data=pax_join)

In [ ]:
pax_join[pax_join.column=='RPLC'].sort_values('rank', ascending=False)